In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.autograd import Variable
from mpl_toolkits.mplot3d import Axes3D
import sklearn.metrics as metrics
import numpy.linalg as la
import pickle
%matplotlib notebook

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 0
fname = fnames[runfile]
bf = binf[runfile]

with open(os.getcwd()+'/minham_'+fname) as f:
    p = pickle.load(f)

path = '/home/daiict/CVShare/Jeni/hts/bioassay-datasets/' + fname + 'red_train.csv'
p_fingerprints = []
labels = []
with open(path) as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:112])
        labels.append(row[-1])

In [3]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
labels = labels[1:]
print(no_examples,ip_dim)
print("total no of 1s",np.sum(p_fingerprints))
print("total no of 0s",no_examples*ip_dim-np.sum(p_fingerprints))

p_fingerprints[(p_fingerprints==0)] = -1

labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0

no_active_ele = (sum(labels2))
labels2 = labels2.astype(int)
print(no_active_ele)

fname

(3423, 112)
('total no of 1s', 25982)
('total no of 0s', 357394)
[ 48.]


'AID362'

In [4]:
p_inactive = p[1]
p_active = p[0]

In [5]:
p_fingerprints[labels2[:,0]==0] = p_fingerprints[labels2[:,0]==0] - p_inactive
p_fingerprints[labels2[:,0] ==1] = p_fingerprints[labels2[:,0] == 1] - p_inactive

In [14]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        #c_train_data = c_fingerprints[indices]
        p_train_data = p_fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_train_data[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(int)
    else:
        train_batch = c_train_data[samples].reshape(batch_size,feat_dim)
        train_batch = train_batch.astype(float)
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [15]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = p_fingerprints[s_ind:e_ind]
    else:
        train_data = c_fingerprints[s_ind:e_ind]
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    #print train_data.shape, labels_val.shape
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  

In [16]:
class b_mlp(nn.Module):
    def __init__(self):
        super(b_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,300)
        self.l2 = nn.Linear(300,300)
        #self.l3 = nn.Linear(1000,500)
        self.l4 = nn.Linear(300,300)
        self.l5 = nn.Linear(300,50)
        self.l6 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        #x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = (self.l6(x))
        
        return x

In [17]:
min_fn = 15
max_fp = 200
maxtmpdiff = -2
cm_list = []
get_model = 1

batch_size = 90

for val_iter in range(1,6):
    print("val iter: ",val_iter)
    
   
    weights_array = [5]
    #weights_array = np.linspace(50,70,10)
    for i,w in enumerate(weights_array): 
        mymlp = b_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-4)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        cost_track = []
        epochs = 2000
        for ep in range(epochs):
            train_batch,target = get_train_batch(batch_size,binary = True,validation_iter = val_iter)
            model_op = mymlp(train_batch)
            #print(model_op.type)
            #print(target.type)
            loss = criterion(model_op,target)
            loss.backward()
            cost_track.append(loss)
            optimizer.step()
            optimizer.zero_grad()

            if(ep%30==29):
                print(loss.data[0])

        plt.plot(range(epochs),cost_track)
        plt.show()
        
        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter,binary = True)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        tmpdiff = tp-fn
        if(tmpdiff >= maxtmpdiff):
            cm_list.append(wcf)
            if(tmpdiff>= maxtmpdiff):
                if(fp <= max_fp):
                    max_fp = fp
                    maxtmpdiff = tmpdiff
                    model_path = os.getcwd() + '/kernel_mac' + fname + '_' + str(get_model)
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)
                    if(get_model<4):
                        get_model = get_model + 1
                    else:
                        break

('val iter: ', 1)
0.517964482307
0.428581923246
0.340609550476
0.268686562777
0.203801721334
0.156224146485
0.121023505926
0.0975580215454
0.0800771638751
0.0625934079289
0.0525837168097
0.0435859113932
0.03607371822
0.0325652994215
0.0265617761761
0.023246653378
0.0208307895809
0.0175854228437
0.0168043766171
0.0143629843369
0.0129594085738
0.0116958077997
0.0104383975267
0.00989801902324
0.00861436780542
0.00838187243789
0.00763905979693
0.0076474850066
0.00701396213844
0.00640992308035
0.00584557838738
0.0054649328813
0.00565512990579
0.00499389693141
0.00478556938469
0.00450491392985
0.00442972732708
0.00429928814992
0.00387533777393
0.00378034915775
0.00340229133144
0.00346342450939
0.00307195447385
0.00328856054693
0.00302573037334
0.0027908324264
0.00266108778305
0.00253994995728
0.0024267276749
0.00232632691041
0.00227648951113
0.00225038803183
0.0021446228493
0.00209877500311
0.00187400449067
0.00187152752187
0.00186821084935
0.00211125472561
0.00164780882187
0.00177618663292


ValueError: need more than 1 value to unpack

In [18]:
cf

array([685])

In [19]:
metrics.confusion_matrix(labels_val,pred_labels).ravel()

array([685])

In [22]:
pred_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,